In [53]:
monster_hitpoints = 51
monster_damage = 9
player_hitpoints = 50
player_mana = 500

from advent.maze import solve_maze

In [ ]:
# This sounds like a BFS to me :D
from typing import NamedTuple

spells = {
    'shield': 113,
    'poison': 173,
    'recharge': 229,
    'drain': 73,
    'magic missile': 53
}

class Status(NamedTuple):
    player_hitpoints: int
    monster_hitpoints: int
    monster_damage: int
    player_mana: int
    shield_turns: int # Sets player armor to 7
    poison_turns: int # deals 3 damage
    recharge_turns: int # gains 101 mana
    player_turn: bool # Whose next turn it is.

    def cast_spell(self, spell: str):
        assert spells[spell] <= self.player_mana
        #assert not self.player_turn
        d = self._asdict()
        d['player_mana'] -= spells[spell]
        if spell == 'shield':
            d['shield_turns'] = 6
        elif spell == 'poison':
            d['poison_turns'] = 6
        elif spell == 'recharge':
            d['recharge_turns'] = 5
        elif spell == 'drain':
            d['player_hitpoints'] += 2
            d['monster_hitpoints'] -= 2
        elif spell == 'magic missile':
            d['monster_hitpoints'] -= 4
        else: raise ValueError(spell)
        return Status(**d)
    
    def can_cast(self, spell):
        mana = spells[spell] < self.player_mana
        if spell == 'shield':
            return mana and self.shield_turns == 0
        elif spell == 'poison':
            return mana and self.poison_turns == 0
        elif spell == 'recharge':
            return mana and self.recharge_turns == 0
        return mana

    def next_turn(self):
        d = self._asdict()
        # Reduce all timers
        if self.shield_turns > 0:
            d['shield_turns'] -= 1
        if self.poison_turns > 0:
            d['poison_turns'] -= 1
            d['monster_hitpoints'] -= 3
        if self.recharge_turns > 0:
            d['recharge_turns'] -= 1
            d['player_mana'] += 101
        if not self.player_turn:
            player_armor = 0 if self.shield_turns == 0 else 7
            d['player_hitpoints'] -= max(1, self.monster_damage - player_armor)

        d['player_turn'] = not self.player_turn
        return Status(**d)
    
    def hard_mode(self):
        # Should be called at the beginning of player turn before casting a spell
        d = self._asdict()
        if not self.player_turn: return Status(**d)
        d['player_hitpoints'] -= 1
        if d['player_hitpoints'] == 0:
            # Kinda dumb, but we just want to make sure that whatever happens
            # The rest of the turn, we can't win anymore
            d['player_hitpoints'] = -999
            d['monster_hitpoints'] = 999
        return Status(**d)

def is_end(status: Status):
    # You can die on the same turn with poison, but in that case you still win
    return status.monster_hitpoints <= 0 # and status.player_hitpoints > 0

start = Status(
    player_hitpoints,
    monster_hitpoints,
    monster_damage,
    player_mana,
    0, 0, 0,
    True
    )

In [ ]:
from typing import Iterator
def adjacent(status: Status) -> Iterator[tuple[Status, int]]:
    #print(status.player_turn)
    if status.player_hitpoints < 0: return
    if not status.player_turn:
        yield status.next_turn(), 0
    # Now we have to cast a spell
    else:
        if HARDMODE: status = status.hard_mode()
        for spell in spells:
            #print(spell, status.can_cast(spell))
            if not status.can_cast(spell): continue
            newstatus = status
            newstatus = newstatus.cast_spell(spell)
            newstatus = newstatus.next_turn()
            #newstatus = newstatus.cast_spell(spell)
            yield newstatus, spells[spell]

HARDMODE = False
_ = solve_maze(start, is_end, adjacent)

  0%|          | 0/1 [00:00<?, ?it/s]

Final path length: 900


In [59]:
HARDMODE = True
_ = solve_maze(start, is_end, adjacent)

  0%|          | 0/1 [00:00<?, ?it/s]

Final path length: 1216
